In [1]:
import threading
import psycopg2
import time

<hr>

In [2]:
# Функція для встановлення з'єднання з базою даних
def connect_db():
    return psycopg2.connect("host=localhost dbname=massive_insert-postgresql user=sazan24 password=UIOP1234")

In [3]:
# Функція для створення таблиці та ініціалізації даних
def create_table():
    connect = connect_db()
    cursor = connect.cursor()

    cursor.execute("CREATE TABLE user_counter (user_id serial PRIMARY KEY, counter integer, version integer);")
    
    cursor.execute("INSERT INTO user_counter (user_id, counter, version) VALUES (1, 0, 0);")
    cursor.execute("INSERT INTO user_counter (user_id, counter, version) VALUES (2, 0, 0);")
    cursor.execute("INSERT INTO user_counter (user_id, counter, version) VALUES (3, 0, 0);")
    cursor.execute("INSERT INTO user_counter (user_id, counter, version) VALUES (4, 0, 0);")

    connect.commit()
    cursor.close()
    connect.close()

<hr>

In [4]:
# Функція для перегляду всіх записів у таблиці даних
def view_table():
    connect = connect_db()
    cursor = connect.cursor()

    cursor.execute("SELECT * FROM user_counter ORDER BY user_id;")
    table = cursor.fetchall()
    print("user_id, counter, version")
    for row in table: print(row)

    connect.commit()
    cursor.close()
    connect.close()

In [5]:
# Функція для видалення таблиці даних "user_counter"
def drop_table():
    connect = connect_db()
    cursor = connect.cursor()

    cursor.execute("DROP TABLE user_counter;")

    connect.commit()
    cursor.close()
    connect.close()

In [6]:
# Функція для виведення значення лічильника з ID
def print_counter(i):
    connect = connect_db()
    cursor = connect.cursor()

    cursor.execute(f"SELECT counter FROM user_counter WHERE user_id = {i};")
    print(f"Final counter value: {cursor.fetchone()[0]}")

    connect.commit()
    cursor.close()
    connect.close()

In [7]:
# Функція для скидання лічильника та версії з ID
def update_counter(i):
    connect = connect_db()
    cursor = connect.cursor()

    cursor.execute(f"UPDATE user_counter SET counter = 0 WHERE user_id = {i};")
    cursor.execute(f"UPDATE user_counter SET version = 0 WHERE user_id = {i};")

    connect.commit()
    cursor.close()
    connect.close()

<hr>

In [8]:
# Функція, яка демонструє сутність проблеми втраченого оновлення
def lost_update():
    connect = connect_db()
    cursor = connect.cursor()
    
    for _ in range(10000):
        counter = cursor.execute("SELECT counter FROM user_counter WHERE user_id = 1;")
        counter = cursor.fetchone()[0] + 1
        cursor.execute(f"UPDATE user_counter SET counter = {counter} WHERE user_id = 1;")
        connect.commit()
        
    cursor.close()
    connect.close()

In [9]:
# Функція, яка адекватно оновлює значення лічильника "на місці"
def inplace_update():
    connect = connect_db()
    cursor = connect.cursor()
    
    for _ in range(10000):
        cursor.execute("UPDATE user_counter SET counter = counter + 1 WHERE user_id = 2;")
        connect.commit()
        
    cursor.close()
    connect.close()

In [10]:
# Функція, яка використовує принцип блокувань на рівні рядків
def rowlevel_locking():
    for _ in range(10000):
        connect = connect_db()
        cursor = connect.cursor()
        
        cursor.execute("SELECT counter FROM user_counter WHERE user_id = 3 FOR UPDATE;")
        counter = cursor.fetchone()[0] + 1
        
        cursor.execute(f"UPDATE user_counter SET counter = {counter} WHERE user_id = 3;")
        connect.commit()
        
        cursor.close()
        connect.close()

In [11]:
# Функція, що використовує механізм керування паралельним доступом
def concurrency_control():
    connect = connect_db()
    cursor = connect.cursor()
    
    for _ in range(10000):
        while True:
            cursor.execute("SELECT counter, version FROM user_counter WHERE user_id = 4;")
            counter, version = cursor.fetchone()
            
            counter += 1
            cursor.execute(f"UPDATE user_counter SET counter = {counter}, version = {version + 1} \
                             WHERE user_id = 4 and version = {version};")
            connect.commit()
            
            count = cursor.rowcount
            if count > 0: break
                
    cursor.close()
    connect.close()

<hr>

In [12]:
if __name__ == "__main__":
    # Створення та огляд таблиці
    create_table()
    view_table()

    # Завдання до виконання
    tasks = [[lost_update, "1. Lost Update"],
             [inplace_update, "2. In-Place Update"],
             [rowlevel_locking, "3. Row-Level Locking"],
             [concurrency_control, "4. Concurrency Control"]]
    
    for i, (task, name) in enumerate(tasks, start=1):
        print(f"\n{'-' * 50}\nExecuting {name}...\n{'-' * 50}")
        update_counter(i)
        
        start = time.time()
        threads = [threading.Thread(target=task) for _ in range(10)]
        
        for thread in threads: thread.start()
        for thread in threads: thread.join()
        
        print(f"Task '{name}' executed in: {time.time() - start:.2f} seconds")
        print_counter(i)
        print(f"{'-' * 50}\n")

    # Перегляд таблиці
    view_table()

user_id, counter, version
(1, 0, 0)
(2, 0, 0)
(3, 0, 0)
(4, 0, 0)

--------------------------------------------------
Executing 1. Lost Update...
--------------------------------------------------
Task '1. Lost Update' executed in: 235.49 seconds
Final counter value: 10559
--------------------------------------------------


--------------------------------------------------
Executing 2. In-Place Update...
--------------------------------------------------
Task '2. In-Place Update' executed in: 258.13 seconds
Final counter value: 100000
--------------------------------------------------


--------------------------------------------------
Executing 3. Row-Level Locking...
--------------------------------------------------
Task '3. Row-Level Locking' executed in: 832.70 seconds
Final counter value: 100000
--------------------------------------------------


--------------------------------------------------
Executing 4. Concurrency Control...
--------------------------------------------